In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import sys
import os

import sklearn
from sklearn import cross_validation
from sklearn import svm
from sklearn import metrics

reload(sys)
#sys.setdefaultencoding("ISO-8859-1")
sys.setdefaultencoding("UTF-8")

def clean_text(raw_text):
    review_text = BeautifulSoup(raw_text).get_text()
    words = review_text.lower().split()
    return(" ".join(words))

In [35]:
import re 

languages = ["english"]
datafiles = ["summary-english-truth.txt"]
tasks = ["age"]

train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
num_text = train["text"].size

clean_train_data = []
urls = []
hashtags = []
for i in xrange( 0, num_text):
    one_clean_line = clean_text( train["text"][i] )
    new_clean_line = ""
    #replacing links
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', one_clean_line)
    for one_url in url:
        new_clean_line = one_clean_line.replace(one_url, "LINK_HERE")
        one_clean_line = new_clean_line
    urls.append(url)
    
    hashtag = re.findall('#(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', one_clean_line)
    
    for one_hashtag in hashtag:
        new_clean_line = one_clean_line.replace(one_hashtag, "HASHTAG_HERE")
        one_clean_line = new_clean_line
        
    urls.append(url)
    hashtags.append(hashtag)
    clean_train_data.append( one_clean_line )

In [37]:
clean_train_data

[u'how to test your startup idea for $50 LINK_HERE @username @username @username @username @username you\'ve been quoted in my HASHTAG_HERE story "new story" LINK_HERE new story LINK_HERE HASHTAG_HERE HASHTAG_HERE @username @username @username @username you\'ve been quoted in my HASHTAG_HERE story HASHTAG_HERE LINK_HERE @username @username @username @username @username you\'ve been quoted in my HASHTAG_HERE story HASHTAG_HERE LINK_HERE @username you\'ve been quoted in my HASHTAG_HERE story HASHTAG_HERE LINK_HERE business models are a commodity stop asking "but how will they make money?": LINK_HERE y combinator alum flutter raises $1.4 million for gesture recognition tech LINK_HERE montreal protests, 4 months in (photos)- the atlantic LINK_HERE "i don\'t want to archieve immortality through my work... i want to achieve it through not dying." woody allen por comer en la oficina d lvk (@username st. andrews) LINK_HERE @username take it easy ! i\'m at aeroparque jorge newbery (aep) (ciudad

In [12]:
#checking if there are still urls
urls = []
for one_clean_line in clean_train_data:
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', one_clean_line)
    urls.append(url)

In [13]:
urls

[[u'http://t.co/jtrdxtnd',
  u'http://t.co/uu5aggzp',
  u'http://t.co/uu5aggzp',
  u'http://t.co/uu5aggzp',
  u'http://t.co/uu5aggzp',
  u'http://t.co/uu5aggzp',
  u'http://t.co/1lx60fs9',
  u'http://t.co/sklqfkds',
  u'http://t.co/svf238vm',
  u'http://t.co/vur3wbk6',
  u'http://t.co/2ey8mvwh',
  u'http://t.co/rgrmddem',
  u'http://t.co/mp1jlnrp',
  u'http://t.co/lelxvepm',
  u'http://t.co/dsf4kblj',
  u'http://t.co/oy7junga',
  u'http://t.co/itp3vnrx',
  u'http://t.co/rlp6mlf8',
  u'http://t.co/yofp2bhu',
  u'http://t.co/4h81dabj',
  u'http://t.co/jdd4cukw',
  u'http://t.co/gmsvlmld',
  u'http://t.co/4qm2q4zc',
  u'http://t.co/bz29s9xw',
  u'http://t.co/3nftyqxs',
  u'http://t.co/umpybbvh',
  u'http://t.co/xxmktwzg',
  u'http://t.co/vof1lokq',
  u'http://t.co/zfysyszl',
  u'http://t.co/pnyeew5w',
  u'http://t.co/r7wanct0',
  u'http://t.co/scfndyyb',
  u'http://t.co/mtsgcl1g',
  u'http://t.co/gil6dkzj',
  u'http://t.co/8pbefsnh',
  u'http://t.co/3vlljzzo',
  u'http://t.co/3tftcudq',
 